# Stock Prediction Demo

In [ ]:
#@title IMPORTS

!pip install streamlit

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np

allCompanies = ["AAPL", "AMZN", "GOOG", "META", "MSFT", "NVDA", "TSLA", "GLNCY", "BHP", "RIO", "VALE", "CSUAY", "NGLOY", "ZIJMF", "WMT", "COST", "HD", "JD", "KR",  "WBA", "BABA", "TGT", "LOW", "LEN", "CAT", "CCS", "MTH", "TOL", "DHI", "KBH", "TM", "BYDDF", "MBGYY", "STLA", "GM", "RACE", "F", "HMC"]

techCompanies = ["AAPL","MSFT","AMZN","TSLA","NVDA", "PYPL"]
df = pd.read_csv(allCompanies[0]+".csv", index_col=0)
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
df["Change"] = (df["Close"] - df["Open"]) / df["Open"] * 100

for i in range(1,len(allCompanies)):
  dfNew = pd.read_csv(allCompanies[i]+".csv", index_col=0)
  dfNew['Open'] = pd.to_numeric(dfNew['Open'], errors='coerce')
  dfNew["Change"] = (dfNew["Close"] - dfNew["Open"]) / dfNew["Open"] * 100
  df = df.merge(dfNew, on="Date", how = "inner", suffixes=["_"+allCompanies[i-1],"_"+allCompanies[i]])

columns = ["Change_"+a for a in allCompanies]
columns.append("Date")
dfPrices = df[columns]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title APP
%%writefile app.py

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np
import streamlit as st
import plotly.express as px

def main():
    st.title("Sunset Capital")

    allCompanies = ["AAPL", "AMZN", "GOOG", "META", "MSFT", "NVDA", "TSLA", "GLNCY", "BHP", "RIO", "VALE", "CSUAY", "NGLOY", "ZIJMF", "WMT", "COST", "HD", "JD", "KR",  "WBA", "BABA", "TGT", "LOW", "LEN", "CAT", "CCS", "MTH", "TOL", "DHI", "KBH", "TM", "BYDDF", "MBGYY", "STLA", "GM", "RACE", "F", "HMC"]

    techCompanies = ["AAPL","MSFT","AMZN","TSLA","NVDA", "PYPL"]
    df = pd.read_csv(allCompanies[0]+".csv", index_col=0)
    df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
    df["Change"] = (df["Close"] - df["Open"]) / df["Open"] * 100

    for i in range(1,len(allCompanies)):
      dfNew = pd.read_csv(allCompanies[i]+".csv", index_col=0)
      dfNew['Open'] = pd.to_numeric(dfNew['Open'], errors='coerce')
      dfNew["Change"] = (dfNew["Close"] - dfNew["Open"]) / dfNew["Open"] * 100
      df = df.merge(dfNew, on="Date", how = "inner", suffixes=["_"+allCompanies[i-1],"_"+allCompanies[i]])
    
    columns = ["Change_"+a for a in allCompanies]
    columns.append("Date")
    dfPrices = df[columns]

    companies = st.text_input('Enter the companies you want to trade, seperated by a single space:')
    str_list = companies.split()
    st.write('Give us a moment to learn about: ', [i for i in str_list])

    divider = len(str_list)
    new_sl = []
    for strin in str_list:
      strin = "Change_" + strin
      new_sl.append(strin)

    
    dfPrices["INDEX"] = 0
    for com in new_sl:
      dfPrices["INDEX"] += dfPrices[com]
    dfPrices["INDEX"] /= len(new_sl)
    train_df = pd.concat([dfPrices.iloc[:, -1].shift(1), dfPrices.iloc[:, :-1]], axis=1)
    train_df = train_df.dropna()
    st.write(train_df)

    train = train_df[238:]
    test = train_df[:238]

    model = RandomForestRegressor(n_estimators=100, random_state=42)

    X_train = train.drop('Date', axis=1)
    X_train = X_train.drop('INDEX', axis=1)
    y_train = train['INDEX']

    model.fit(X_train, y_train)



    X_test = test.drop('Date', axis=1)
    X_test = X_test.drop('INDEX', axis=1)
    y_test = test['INDEX']
    y_pred = model.predict(X_test)

    # Calculate the proportion of instances where both y_test and y_pred are either greater than zero or less than zero
    positive_positive = ((y_test > 0) & (y_pred > 0)).sum() / len(y_test)
    negative_negative = ((y_test < 0) & (y_pred < 0)).sum() / len(y_test)

    pvt = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})
    pvt["positive"] = (pvt["y_pred"] > 0) * 1 
    pvt["bankroll_movement"] = ((pvt["y_test"] * pvt["positive"]) / 100) + 1

    revenue_changes = pvt["bankroll_movement"].tolist()
    var = 1
    revenue_changes
    bankroll = []

    for i in range(len(revenue_changes)):
      var *= revenue_changes[i]
      bankroll.append(var)

    st.write(f"Overall yearly bankroll change: {(var *100) - 100:.2f}%")
    st.write("In this time, the S&P 500 is -12.94%")
    st.write(bankroll)

    
    px.line(bankroll)

if __name__ == "__main__":
    main()


Overwriting app.py


# RUN APP

In [44]:
!streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 4.599s



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.201.229.133:8501

your url is: https://solid-ants-rhyme-35-201-229-133.loca.lt
/content/app.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPrices["INDEX"] = 0
/content/app.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPrices["INDEX"] /= len(new_sl)
2023-03-22 00:46:26.976 Uncaught app exception
Traceback (most recent call last):
  File "/usr